# Chapter 1: Where in the genome does DNA replication begin?

## Counting Words

> We use the term **k-mer** to refer to a string of length *k* and define COUNT(Text, Pattern) as the number of times that a k-mer *Pattern* appears as a substring of *Text*

Below, implement PATTERNCOUNT(*Text*, *Pattern*) as a sliding window over *Text* checking whether *Pattern* appears as a substring. Return the number of times *Pattern* appears as a substring in *Text*

In [1]:
def pattern_count(text: str, pattern: str) -> int:
    count = 0
    k = len(pattern)
    for i in range(len(text) - k + 1):
        if text[i:i+k] == pattern:
            count += 1
            
    return count

# test data - should return 3 -------------------------------------------------
s = 'ACAACTATGCATACTATCGGGAACTATCCT'
p = 'ACTAT'

print(pattern_count(s, p))

3


## Frequent Words

> A straightforward algorithm for finding the most frequent k-mers in a string *Text* checks all k-mers appearing in this string then computes how many times each k-mer appears in *Text*

Below, implement the inefficient FREQUENTWORDS(*Text*, *k*) algorithm. Return the most frequent k-mers in text.

In [2]:
from typing import Set


def frequent_words(text: str, k: int) -> Set:
    frequent_patterns = set()
    count = []
    for i in range(len(text) - k + 1):
        pattern = text[i:i+k]
        count.append(pattern_count(text, pattern))
    
    max_count = max(count)
    for i in range(len(text) - k + 1):
        if count[i] == max_count:
            frequent_patterns.add(text[i:i+k])
            
    return frequent_patterns

# -----------------------------------------------------------------------------
Text = 'ACTGACTCCCACCCC'
K = 3

print(frequent_words(Text, K))

{'CCC'}


## Building a faster frequent words function

This implementation slides down *Text* only once, adding a count to the frequency array of a string *Text* where the ith element of the array holds the count of the number of times that ith k-mer appears in *Text*

In [3]:
from typing import List


def pattern_to_number(pattern: str) -> int:
    if not pattern:
        return 0
    
    symbols = {"A":0, "C":1, "G":2, "T":3}
    symbol = pattern[-1]
    prefix = pattern[:-1]
    
    return 4 * pattern_to_number(prefix) + symbols[symbol]


def number_to_pattern(index: int, k: int) -> str:
    numbers = "ACGT"
    
    if k == 1:
        return numbers[index]
    
    prefix_index = index // 4
    r = index % 4
    symbol = numbers[r]
    prefix_pattern = number_to_pattern(prefix_index, k-1)
    
    return prefix_pattern + symbol


def computing_frequencies(text: str, k: int) -> List:
    frequency_array = [0 for _ in range(4**k)]
    for i in range(len(text) - k + 1):
        pattern = text[i:i+k]
        j = pattern_to_number(pattern)
        frequency_array[j] += 1
    
    return frequency_array


def faster_frequent_words(text: str, k: int) -> Set:
    frequent_patterns = set()
    frequency_array = computing_frequencies(text, k)
    max_count = max(frequency_array)
    for i in range(4**k-1):
        if frequency_array[i] == max_count:
            pattern = number_to_pattern(i, k)
            frequent_patterns.add(pattern)
    
    return frequent_patterns


faster_frequent_words(Text, K)

{'CCC'}

## Finding Frequent Words by Sorting

In [4]:
def frequent_words_by_sorting(text: str, k: int) -> Set:
    frequent_patterns = set()
    index = [0 for _ in range(len(text) - k + 1)]
    count = [0 for _ in index]
    for i in range(len(text) - k + 1):
        pattern = text[i:i+k]
        index[i] = pattern_to_number(pattern)
        count[i] += 1
    
    sorted_index = sorted(index)
    for i in range(1, len(text) - k + 1):
        if sorted_index[i] == sorted_index[i-1]:
            count[i] = count[i-1] + 1
    
    max_count = max(count)
    for i in range(len(text) - k + 1):
        if count[i] == max_count:
            pattern = number_to_pattern(sorted_index[i], k)
            frequent_patterns.add(pattern)
    
    return frequent_patterns        

frequent_words_by_sorting(Text, K)

{'CCC'}

## Pythonic Frequent Words

Instead of using two arrays, implement the algorithm using Python standard library functions. Using a `defaultdict` will allow a single pass through *Text* while adding unobserved k-mers to a dictionary. After all k-mers and their counts have been added to the dictionary, return the keys (words) with the max count.

In [5]:
from collections import defaultdict


def frequent_words_pythonic(text: str, k: int) -> List:
    words = defaultdict(int)
    for i in range(len(text) - k + 1):
        pattern = text[i:i+k]
        words[pattern] += 1
    
    max_count = max(words.items(), key=lambda x: x[1])
    frequent_words = [key for key, value in words.items() if value == max_count[1]]
            
    return frequent_words

frequent_words_pythonic(Text, K)

['CCC']

## Finding the reverse complement of a DNA string

In [6]:
def reverse_complement(pattern: str) -> str:
    transtable = str.maketrans('ACGT', 'TGCA')
    return pattern.translate(transtable)[::-1]

reverse_complement(Text)

'GGGGTGGGAGTCAGT'

## Clump Finding

Slide a window of length *L* along the genome, looking for a region where a k-mer appears several times in short succession. Given integers *L* and *t*, a k-mer forms an **(*L*, *t*)-Clump** inside a longer string *Genome* if there is an interval of *Genome* of length *L* in which this k-mer appears at least *t* times.

Below, use `BioPython` to get a sample Genome of *Salmonella enterica* from Entrez 

In [7]:
from Bio import Entrez
from Bio import SeqIO


# Use the optional email parameter
Entrez.email = "jcalendo@gmail.com"

# retrieve the Salmonella Enterica record 
handle = Entrez.efetch(db="nucleotide", id="AE006468.2", rettype="gb", retmode="text")

# read into a SeqRecord
record = SeqIO.read(handle, "genbank")
sequence = record.seq

The implementation below is very slow. 

In [ ]:
def clump_finding(genome: str, k: int, L: int, t: int) -> Set:
    frequent_patterns = set()
    clump = [0 for _ in range(4**k)]

    for i in range(len(genome) - L + 1):
        text = genome[i:i+L]
        frequency_array = computing_frequencies(text, k)
        for f in frequency_array:
            if f >- t:
                clump[j] = 1
    
    for c in clump:
        if c == 1:
            pattern = pattern_to_number(i, k)
            frequent_patterns.add(pattern)
    
    return frequent_patterns


clump_finding(genome=sequence, k=9, L=500, t=3)